In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        # 매우 큰 데이터 구조체를 표현하는 문자열을 축약해서 생성한다. (30자 제한) 
        return "Sentence(%s)" % reprlib.repr(self.text)    

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
# Sentence 객체는 반복할 수 있다.
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
# 반복할 수 있으므로 Sentence 객체는 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용할 수 있다. 
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

---

In [9]:
RE_WORD = re.compile('\w+')
RE_WORD.findall("The King of Passion123 asd 214@#!#124ipjdlkasmk")

['The', 'King', 'of', 'Passion123', 'asd', '214', '124ipjdlkasmk']

In [10]:
class Foo:
    def __iter__(self):
        pass

In [11]:
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [12]:
f = Foo()
isinstance(f, abc.Iterable)

True

## 14.2 반복형과 반복자

In [13]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [14]:
s = 'ABC'
it = iter(s)

while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [19]:
from collections import Iterable
from abc import abstractmethod 

class Iterator(Iterable):
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
               any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
                
        return NotImplemented

In [20]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)   # 반복자를 가져온다. 
it

In [21]:
next(it)

'Pig'

In [22]:
next(it)

'and'

In [23]:
next(it)

'Pepper'

In [26]:
next(it)   # 더 이상 단어가 없으므로 예외를 발생시킨다. 

StopIteration: 

In [24]:
list(it)  # 소진된 후에는 반복자가 필요 없다. 

[]

In [25]:
list(iter(s3))   # 다시 반복하려면 생성자를 새로 만들어야 한다. 

['Pig', 'and', 'Pepper']

## 14.3 고전적인 반복자

In [30]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return "Sentence(%s)" % reprlib.repr(self.text)    
    
    def __iter__(self):    # 반복자 객체를 생성해서 반환함으로써 반복형 프로토콜 구현
        return SentenceIterator(self.words) 
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words    # 단어 리스트에 대한 참조를 담고 있다. 
        self.index = 0    # 다음에 가져올 단어를 결정하기 위해 self.index를 사용한다. 
        
    def __next__(self):
        try:
            word = self.words[self.index]    # self.index 에 있는 단어를 가져온다, 
        except IndexError:
            raise StopIteration()    # self.index 에 단어가 없으면 예외 발생
        self.index += 1    # 인덱스 증가 
        return word    # 단어 반환
    
    def __iter__(self):
        return self    

In [31]:
s = Sentence('"The time has come," the Walrus said,')
# Sentence 객체는 반복할 수 있다.
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [32]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']